## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [ ]:
tracks = session.query(Track).limit(3).all()

for t in tracks:
    print(t.TrackId, t.Name, t.AlbumId, t.UnitPrice)

In [ ]:
result = (
    session.query(Track.Name, Album.Title)
    .join(Album, Track.AlbumId == Album.AlbumId)
    .limit(20)
    .all()
)

for row in result:
    print(row)


In [ ]:
sales = session.query(InvoiceLine).limit(10).all()

for s in sales:
    print(s.InvoiceLineId, s.TrackId, s.Quantity, s.UnitPrice)

result = (
    session.query(Track.Name, InvoiceLine.Quantity)
    .join(InvoiceLine, Track.TrackId == InvoiceLine.TrackId)
    .limit(10)
    .all()
)

for row in result:
    print(row)



In [ ]:
top_tracks = (
    session.query(
        Track.Name,
        func.sum(InvoiceLine.Quantity).label("total_vendido")
    )
    .join(InvoiceLine, Track.TrackId == InvoiceLine.TrackId)
    .group_by(Track.TrackId)
    .order_by(func.sum(InvoiceLine.Quantity).desc())
    .limit(10)
    .all()
)

for row in top_tracks:
    print(row)


In [ ]:
top_artists = (
    session.query(
        Artist.Name,
        func.sum(InvoiceLine.Quantity).label("total_vendido")
    )
    .join(Album, Artist.ArtistId == Album.ArtistId)
    .join(Track, Album.AlbumId == Track.AlbumId)
    .join(InvoiceLine, Track.TrackId == InvoiceLine.TrackId)
    .group_by(Artist.ArtistId)
    .order_by(func.sum(InvoiceLine.Quantity).desc())
    .limit(10)
    .all()
)

for row in top_artists:
    print(row)
